# Fourier transformation

This notebook is responsible for converting the eeg data stored in the database into images representing a second of data.

The data is transormed using fourier transformation, and each timeframe represents a row in the final image. To accomodate for the complex numbers produces using fourier transformation, the real part is placed in the red channel of the image, and the imaginary part is placed in the green. The blue channel is unused and is filled with zero. 

To make a simple test on training with this data, the dataset is turned into a classification problem to see if the pasient is working with their left or right side, this is divided into 4 labels

1. none - used for baseline recordings, where they are instructed to lie still
2. left - they move their left hand
3. right - they move their right hand
4. both - they move both hands or feet

In [5]:
import sql_data_manager as sql
import torch
import torchvision
from tqdm import tqdm

In [6]:
database = sql.SqlDataSet("no_baseline_model.db")

In [7]:


REST_POSITION = 1

OPEN_CLOSE_LEFT_FIST_ACTUAL = 2
OPEN_CLOSE_LEFT_FIST_IMAGINED = 3
OPEN_CLOSE_RIGHT_FIST_ACTUAL = 4
OPEN_CLOSE_RIGHT_FIST_IMAGINED = 5
OPEN_CLOSE_BOTH_FISTS_ACTUAL = 6
OPEN_CLOSE_BOTH_FISTS_IMAGINED = 7
OPEN_CLOSE_BOTH_FEET_ACTUAL = 8
OPEN_CLOSE_BOTH_FEET_IMAGINED = 9

def is_both(v: torch.Tensor) -> bool:
    return OPEN_CLOSE_BOTH_FISTS_ACTUAL/9.0 in v or OPEN_CLOSE_BOTH_FEET_ACTUAL/9.0 in v 

def is_left(v: torch.Tensor) -> bool:
    return OPEN_CLOSE_LEFT_FIST_ACTUAL/9.0 in v or is_both(v)

def is_right(v: torch.Tensor) -> bool:
    return OPEN_CLOSE_RIGHT_FIST_ACTUAL/9.0 in v or is_both(v)

In [8]:
from os import makedirs


data = []
window_size = 160
# good overlap
step_size = window_size // 3

made_dirs = set()
for recording in tqdm(range(len(database))):
    signals, labels = database[recording]

    for chunk in range(0, signals.shape[0], step_size):
        start = chunk*step_size
        end = chunk*step_size + window_size
        view = signals[start:end, :, :]
        if view.shape[0] != 160:
            continue
        
        label_view = labels[start:end, :]

        transformed = torch.fft.fft(view.view(-1,64))
        red = transformed.real.view(1, -1, 64,)
        green = transformed.imag.view(1, -1, 64,)
        blue = torch.zeros(1, transformed.shape[0], 64,)
        
        merged = (torch.cat((red, green, blue), 0)*255).byte()

        if merged.shape[1] < 160:
            print(merged.shape)
        #merged = torch.nn.functional.pad(merged, ())

        left = is_left(label_view)
        right = is_right(label_view)

        label = ""
        match (left, right):
            case (False, False):
                label = "none"
            case (True, False):
                label = "left"
            case (False, True):
                label = "right"
            case (True, True):
                label = "both"

        dir = f"./data/generated/{label}"
        filename = f"{dir}/r{recording}o{chunk}.png"
        if dir not in made_dirs:
            made_dirs.add(dir)
            makedirs(dir, exist_ok=True)

        torchvision.io.write_png(merged, filename)

        # plt.imshow(transformed[0,:].real)
        # plt.figure()
        # plt.imshow(transformed[0,:].imag)

100%|██████████| 1296/1296 [01:04<00:00, 20.11it/s]
